# Guided Studies into Financial Management
## Index Revisions and Stock Returns

### Colaborators
Dennis Blaufuss,
Lars Wrede,
Nicolas Kepper,
Sophie Merl,
Philipp Voit

### Instructor
Dr. Stefan Scharnoski

### Summary 
HIER MÜSSEN WIR NOCH EINE ZUSAMMENFASSUNG DER ERGEBNISSE SCHREIBEN - WIE ABSTRACT

---

## Table of Content:
1. [Data Proprocessing](#0-bullet)
* [Import Data](#first-bullet)
* [Calculate Daily Returns](#2-bullet)
* [Data Quality Checks](#3-bullet)
* [Descriptive Statistics](#4-bullet)
2. [Price Pressure](#5-bullet)
3. [Investor Attention](#6-bullet)
4. [Systematic Risk and Liquidity](#7-bullet)
* [Systematic Risk - Included Stocks](#8-bullet)
* [Systematic Risk - Excluded Stocks](#9-bullet)
* [Systematic Risk - DAX 30 to DAX 40](#10-bullet)
___

# Change to code cell if packages not installed yet
import sys  
!{sys.executable} -m pip install yfinance  
!{sys.executable} -m pip install performanceanalytics  

In [6]:
import pandas as pd
import numpy as np 
import datetime

import yfinance as yf

import performanceanalytics.table.table as pat
from performanceanalytics.charts import performance_summary
import statistics
import statsmodels.formula.api as smf

# 1. Data Proprocessing <a class="anchor" id="0-bullet"></a>

## Import Data <a class="anchor" id="first-bullet"></a>

In [9]:
''' Import all relevant data.
Parameters
----------
:DAX_aufgenommen: df
    All stocks included in the DAX from 2010 until 2021.
:DAX_ausgeschieden: df
    All stocks excluded from the DAX from 2010 until 2021.
:stock_data: df
    Contains stock ticker as well as names of the stocks of all DAX stock from 22-03-01.
:benchmark:  df
    The MSCI Germany Index was used as a proxy for the market portfolio. 
    This index contains a large number of M-DAX and DAX stocks and is therefore more 
    broadly structured than the DAX. However, the high weight of the DAX shares in the index is problematic, 
    so that it is to be expected that the actual influence is underestimated.
:index_compositions: df
    Contains the deletions/ additions as well as date of change/ announcements & Merger/Spin-Off Information.
-------
'''
raw_data_dir = '../01_RawData/'
DAX_included = pd.read_csv(raw_data_dir + 'DAX_included_2010-2021.csv', sep = ';')
DAX_excluded = pd.read_csv(raw_data_dir + 'DAX_excluded_2010-2021.csv', sep = ';')
stock_data = pd.read_csv(raw_data_dir + 'Companies_Ticker.csv', sep = ';')
benchmark = pd.read_csv(raw_data_dir + 'DAX_Kurs.csv', sep = ';')
#index_compositions = pd.read_csv('Historical_Index_Compositions.csv', sep = ';')

In [10]:
''' Pulls time series data for stocks on a daily basis from 2010-01-01 until 2022-03-01.

Parameters
----------
:stock_dict:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_dict = {}
for s in stock_data['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_dict[s] = yf.download(s, start='2010-01-01', end='2022-03-01', progress=False)
# Concatenate all data
stocks_as_df = pd.concat(stock_dict, axis = 0)

In [11]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_included = {}
for s in DAX_included['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_included[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_included_as_df = pd.concat(stock_included, axis = 0)

KeyError: 'Symbol'

In [ ]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_excluded = {}
for s in DAX_excluded['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_excluded[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_excluded_as_df = pd.concat(stock_excluded, axis = 0)

## Calculate Daily Returns <a class="anchor" id="2-bullet"></a>

In [ ]:
''' Transform daily price data to daily returns
Parameters
----------
:returns_daily:  dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_excluded: dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_included: dict
    Contains the stock symbols as key and the daily returns as values.
:benchmark: df
    Contains daily returns from the benchmark.
-------
'''
returns_daily = {}
for s in stock_data['Symbol']:
    returns_daily[s] = stock_dict[s]['Adj Close'].pct_change()
returns_daily_included = {}
for s in DAX_included['Symbol']:
    returns_daily_included[s] = stock_included[s]['Adj Close'].pct_change()
DAX_included['Aufgenommen'] = pd.to_datetime(DAX_included['Aufgenommen'], format='%d.%m.%Y')

returns_daily_excluded = {}
for s in DAX_excluded['Symbol']:
    returns_daily_excluded[s] = stock_excluded[s]['Adj Close'].pct_change()
DAX_excluded['Ausgeschieden'] = pd.to_datetime(DAX_excluded['Ausgeschieden'], format='%d.%m.%Y')
    
benchmark['Umtauschdatum'] = pd.to_datetime(benchmark['Umtauschdatum'], format='%d.%m.%y')
benchmark = pd.DataFrame(benchmark['Schlusskurs'].astype(float).pct_change()).set_index(benchmark['Umtauschdatum'])

## Data Quality Checks <a class="anchor" id="3-bullet"></a>

In [ ]:
'''Check if stocks_as_df contains NA or zeros in Volume & Adjusted Close

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_Volume_is_0:  df
    Contains the rows where Volume == 0.
-------
'''

stocks_as_df_has_nan = np.isnan(np.sum(stocks_as_df))

#(stocks_as_df < 0).any()
# (stocks_as_df = 0).any()

stocks_as_df_Volume_is_0 = stocks_as_df.loc[stocks_as_df["Volume"] == 0]

In [ ]:
'''Check if Adj Close in stocks_as_df differs from previous/ following day.

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_adjclose_peak_bottom:  dataframe
    Contains the rows where Adj. Close differs
-------
'''
stocks_as_df_adjclose_peak_bottom_list = []
n = 1

while n < len(stocks_as_df)-1:
    if abs(stocks_as_df["Adj Close"][n] -
           statistics.mean([stocks_as_df["Adj Close"][n-1],
                            stocks_as_df["Adj Close"][n+1]])) > .5 * stocks_as_df["Adj Close"][n]:
        stocks_as_df_adjclose_peak_bottom_list.append(stocks_as_df.iloc[n])

    n += 1

stocks_as_df_adjclose_peak_bottom = pd.DataFrame(stocks_as_df_adjclose_peak_bottom_list)

## Descriptive Statistics of the whole Dataset <a class="anchor" id="4-bullet"></a>

In [ ]:
''' Calculating measures of location, statistical dispersion and shape
Parameters
----------
:des_stat:  dataframe
    Contains the descriptive statistics.
-------
'''

des_stat = pd.DataFrame(columns=stock_data['Symbol'], 
                        index=['Observations', 'NAs', 'Minimum', 'Quartile 1', 'Median', 
                               'Artithmetic Mean', 'Geometric Mean', 'Quartile 3', 'Maximum', 'SE Mean',
                               'LCL Mean (.95)', 'UCL Mean (.95)', 'Variance', 'Stdev', 'Skewness','Kurtosis'])

for s in stock_data['Symbol']:
    df = pd.DataFrame(returns_daily[s])
    des_stat[s] = pat.stats_table(df, manager_col=0)
des_stat

In [ ]:
''' Calculating the downside statistics
Parameters
----------
:down_stat:  dataframe
    Contains the downside statistics.
-------
'''
#down_stat = pd.DataFrame(columns=stock_data['Symbol'], 
#                        index=['Semi Deviation', 'Gain Deviation', 'Loss Deviation', 'Downside Deviation (MAR=2.0%)',
#                               'Downside Deviation (rf=0.5%)', 'Downside Deviation (0%)', 'Maximum Drawdown', 
#                               'Historical VaR (95%)', 'Historical ES (95%)', 'Modified VaR (95%)', 'Modified ES (95%)'])

#for s in stock_data['Symbol']:
#    df = pd.DataFrame(returns_daily[s])
#    down_stat[s] = pat.create_downside_table(df,0)
#down_stat

___
## 2. Price Pressure <a class="anchor" id="5-bullet"></a>

__To-Do's__
* Welche Index Inklusion Effekte gibt es.
* Gibt es einen Pre-announcement drift? 
* Gibt es sonstige Announcement Effekte?

__Paper 1 EMH vs. PPH nachbauen__
* Alle Aktien für die Analyse zusammen suchen - bisher sind nur die neusten 40 enthalten (Sophie hat hier schon eine CSV vorbereitet).
*    Analyse excess return & trading volume on the first 5 days with the cross sectional means.
*    Analysis before and after annoucment as well as after the inclusion day

* Vorschläge hierzu von Stefan:
    * Alles Herausnahmen und Hereinnahmen in den DAX zusammennehmen, da nur so statistische Test möglich sind
    * Bspw. vor 2010, nach 2010 Veränderungen anschauen





In [ ]:
stocks_included_as_df

Änderungsvorschläge von Stefan
	•	Alles Änderungen zusammen (30 auf 40)
--> inferenzen, statistischen tests nur so möglich
	•	Vor 2010, nach 2010
	•	Überlegen, wie wir das empirisch machen --> counterfactual
	•	Abnormal returns, worauf basiert es; was ist expected return; komplexeres modell
	•	Index Inklusion Effekte
	•	Freiheitsgrade im empirischen Ansatz, solange die Frage beantwortet wird wie sich Inklusion auf Expected Returns und andere Metriken auswirkt
	•	Abnormal returns; Volatilität; Handelsvolumen (ETFs müssen sie nun auch handeln); Investors attention (das sollten wir absprechen; ggf. Aufnahme in Index als Maß für Attention; dazu: googeln)
	•	Korrelationen: Wie ändern sich Korrelation (das könnte attention sein) --> ökonomisch bedeutsam, weil systematisches Risiko
	•	Ggf. Datenfiltern; Daten Fehler; für Preise funktioniert gut; Handelsvolumen nicht so zuverlässig für damals
	•	Wichtig, dass Tage stimmen, ansonsten problematisch
	•	Announcement day & inclusion day
	•	Pre-announcement drift
	•	Announcement Effekte
	•	Dividends announcements --> Literatur

___
## 3. Investor Attention oder Correlation Analysis <a class="anchor" id="6-bullet"></a>

__To-Do's__ 
* Welche Index Inklusion Effekte gibt es.
* Gibt es einen Pre-announcement drift? 
* Gibt es sonstige Announcement Effekte?

__Paper 2 Investor Attention am Ende nachbauen__
* Korrelationen: Wie ändern sich Korrelation (das könnte attention sein) --> ökonomisch bedeutsam, weil systematisches Risiko --> Hinführung zum 4. Teil

___
## 4. Systematic Risk and Liquidity <a class="anchor" id="7-bullet"></a>

It is examined whether the inclusion of a share in the DAX affects the systematic risk and the liquidity of the share in question.

## Systematic Risk of all newly Stocks included in the DAX <a class="anchor" id="8-bullet"></a>

In [11]:
''' Calculating the systmatic risk after the inclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 0
j = 1
sys_risk = []
while i in range(0,10):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
  
while i in range(10,len(returns_daily_included)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,HEI.DE,1.0,0.103273,0.496684,0.379531
1,CON.DE,2.0,-0.321822,0.002120,0.634464
2,LXS.DE,3.0,-0.454091,0.000233,0.506027
3,VNA.DE,4.0,0.118351,0.302192,0.302195
4,PSM.DE,5.0,0.055706,0.498706,0.490919
5,1COV.DE,6.0,0.543596,0.002148,0.390857
6,WDI.HM,7.0,0.519190,0.095540,0.177506
7,MTX.DE,8.0,0.089401,0.509244,0.337730
8,DWNI.DE,9.0,-0.008597,0.956859,0.124476
9,DHER.DE,10.0,-0.104012,0.639293,0.085454


## Systematic Risk of all newly Stocks excluded in the DAX <a class="anchor" id="9-bullet"></a>

In [12]:
''' Calculating the systmatic risk after the exclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''

i = 0
j = 1
sys_risk = []
while i in range(0,7):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]   
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
    
while i in range(7,len(returns_daily_excluded)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()            
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,SZG.DE,1.0,0.009832,0.919350,0.516948
1,LXS.DE,2.0,0.130526,0.114502,0.565313
2,SDF.DE,3.0,0.209893,0.231217,0.233359
3,PSM.DE,4.0,-0.318287,0.178126,0.128139
4,CBK.DE,5.0,0.566617,0.000897,0.303567
5,TKA.DE,6.0,0.450349,0.015504,0.282978
6,LHA.DE,7.0,0.090756,0.728232,0.403611
7,WDI.HM,8.0,-0.093844,0.865141,0.015965
8,BEI.DE,9.0,-0.218337,0.014602,0.283706
9,DWNI.DE,10.0,0.460957,0.027038,0.059404


## Systematic Risk of the 10 Stocks from DAX Increase in 2021 <a class="anchor" id="10-bullet"></a>

In [13]:
''' Creating a list and dictionary with all 10 newly added DAX stocks.
Parameters
----------
:newcomers:  list
    Contains the names of the stocks.
:dax_new: dict
    Contains the daily returns of the 10 new stocks.
-------
'''
newcomers = ['AIR.DE', 'SHL.DE', 'ZAL.DE', 'SY1.DE', 'SRT3.DE',  'POAHY', 'HFG.DE', 'BNR.DE', 'QIA.DE', 'PUM.DE']
dax_new = {new: returns_daily[new] for new in newcomers}

In [14]:
''' Creating the dummy variable - 0 before the inclusion day (2021-09-20) and 1 thereafter.
Parameters
----------
:benchmark:  df
    Contains daily returns as well as the dummy variable.
-------
'''
d = []
for date in benchmark.index:
    if str(date) < '2021-09-20 00:00:00':
        d.append(0)
    else: d.append(1)
benchmark['Dummy'] = d

In [15]:
''' Calculating the systmatic risk.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 1 Year before the inclusion day (2020-09-20)
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 1
sys_risk = []
for key in dax_new:
    data = pd.DataFrame(dax_new[key]['2020-09-20':'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs']['2020-09-20':'2022-03-01']
    data['Dummy'] = benchmark['Dummy']['2020-09-20':'2022-03-01']
    stocks_as_df['Volume'][key]['2020-09-20':'2021-09-20']    
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': key,
            'Rank': i, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,AIR.DE,1.0,-0.115361,0.687517,0.428007
1,SHL.DE,2.0,0.139242,0.287608,0.133441
2,ZAL.DE,3.0,0.187244,0.580780,0.065300
3,SY1.DE,4.0,0.296144,0.061525,0.078697
4,SRT3.DE,5.0,0.498162,0.104634,0.033886
5,POAHY,6.0,0.042090,0.842130,0.290775
6,HFG.DE,7.0,0.886685,0.047033,0.035359
7,BNR.DE,8.0,0.203588,0.216613,0.352506
8,QIA.DE,9.0,0.222901,0.348398,0.006456
9,PUM.DE,10.0,0.069367,0.610297,0.274040


In [16]:
'''Distribution of the shares with a higher unit share in the DAX and all those with a weighting of < 1 %. 
Parameters
----------
:des_stat:  df
    N: Number of stocks sorted after the index weight.
    Mean: Mean systematic risk. 
    R^2: Mean R-squared of the model.
-------
'''
des_stat = []
des_stat.append(
        {
            'N': '1-5',
            r"$\varnothing$": sys_risk[:5][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[:5][r"$R^{2}$"].mean()
        }
    )
des_stat.append(
        {
            'N': '6-10',
            r"$\varnothing$": sys_risk[5:][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[5:][r"$R^{2}$"].mean()
        }
    )
pd.DataFrame(des_stat)

,N,$\varnothing$,$R^{2}$
0,1-5,0.201086,0.147866
1,6-10,0.284926,0.191827
